In [1]:
import pandas as pd
import numpy as np

In [2]:
dtypes_dict = {'created':'str',  'ref_type': 'category', 'device_brand': np.float32, \
                'device_countrycode': 'category', 'device_language': 'category', \
                'ref_hash': 'category', 'kind': 'str', 'user_agent': 'str'}

installs = pd.read_csv('data/installs.csv.gzip', dtype=dtypes_dict, low_memory=False)

installs = installs.drop('device_countrycode', axis=1)
installs['created'] = pd.to_datetime(installs['created'], format='%Y-%m-%d %H:%M:%S')

In [3]:
installs = installs.drop(columns=['device_language', 'ip_address', 'event_uuid', 'device_brand', \
                         'device_model', 'click_hash', 'session_user_agent'])

In [4]:
# Normalizacion de valores a binario

installs['wifi'] = installs['wifi'].map({True: 1, False: 0})
installs['wifi'] = installs['wifi'].fillna(-1)

installs['attributed'] = installs['attributed'].map({True: 1, False: 0})
installs['implicit'] = installs['implicit'].map({True: 1, False: 0})

In [5]:
installs['ref_type_android'] = (installs['ref_type'].astype(str).str.contains("1891515180541284343") == True).astype(int)
installs['ref_type_iphone'] = (installs['ref_type'].astype(str).str.contains("1494519392962156891") == True).astype(int)

installs = installs.drop('ref_type', axis=1)

In [6]:
# features de dias

installs['weekday'] = installs['created'].dt.weekday

installs = pd.get_dummies(installs, columns=['weekday'])

# installs = installs.rename(index=str, columns={"weekday_0": "lunes", "weekday_1": "martes", "weekday_2": "miercoles", \
#                              "weekday_3": "jueves", "weekday_4": "viernes", "weekday_5": "sabado", \
#                              "weekday_6": "domingo"})

In [7]:
# feature horarios

bins = [0, 6, 13, 17, 25]
labels = ['morning','afternoon','evening','night']
hours = installs['created'].dt.hour
installs['time'] = pd.cut(hours-5+24 *(hours<5),bins=bins,labels=labels,right=False)

installs = pd.get_dummies(installs, columns=['time'])

In [8]:
# installs kinds to features, creo que no sirve

installs['kind_open'] = (installs['kind'].astype(str).str.lower().str.contains("open|abertura") == True).astype(int)
installs['kind_home'] = (installs['kind'].astype(str).str.lower().str.contains("home") == True).astype(int)
installs['kind_start'] = (installs['kind'].astype(str).str.lower().str.contains("start") == True).astype(int)
installs['kind_view'] = (installs['kind'].astype(str).str.lower().str.contains("view") == True).astype(int)
installs['kind_sign'] = (installs['kind'].astype(str).str.lower().str.contains("sign|registration|account") == True).astype(int)
installs['kind_login'] = (installs['kind'].astype(str).str.lower().str.contains("login|begin") == True).astype(int)
installs['kind_launch'] = (installs['kind'].astype(str).str.lower().str.contains("launch") == True).astype(int)
installs['kind_install'] = (installs['kind'].astype(str).str.lower().str.contains("install") == True).astype(int)
installs['kind_purchase'] = (installs['kind'].astype(str).str.lower().str.contains("purchase|buy") == True).astype(int)
installs['kind_add'] = (installs['kind'].astype(str).str.lower().str.contains("add") == True).astype(int)

In [9]:
# user_agent features

installs['agent_zaful'] = (installs['user_agent'].astype(str).str.lower().str.contains("zaful") == True).astype(int)
installs['agent_cornershop'] = (installs['user_agent'].astype(str).str.lower().str.contains("cornershop") == True).astype(int)
installs['agent_binomo'] = (installs['user_agent'].astype(str).str.lower().str.contains("binomo") == True).astype(int)
installs['agent_postmates'] = (installs['user_agent'].astype(str).str.lower().str.contains("postmates") == True).astype(int)

installs['agent_tiktok'] = (installs['user_agent'].astype(str).str.lower().str.contains("tiktok") == True).astype(int)
installs['agent_sindelantal'] = (installs['user_agent'].astype(str).str.lower().str.contains("sindelantal") == True).astype(int)
installs['agent_gardenscapes'] = (installs['user_agent'].astype(str).str.lower().str.contains("gardenscapes") == True).astype(int)
installs['agent_dalvik'] = (installs['user_agent'].astype(str).str.lower().str.contains("dalvik") == True).astype(int)
installs['agent_wish'] = (installs['user_agent'].astype(str).str.lower().str.contains("wish") == True).astype(int)
installs['agent_trivago'] = (installs['user_agent'].astype(str).str.lower().str.contains("trivago") == True).astype(int)
installs['agent_grability'] = (installs['user_agent'].astype(str).str.lower().str.contains("grability") == True).astype(int)
installs['agent_mercadopago'] = (installs['user_agent'].astype(str).str.lower().str.contains("mercadopago") == True).astype(int)

In [10]:
# installs['user_agent'].loc[~installs['user_agent'].astype(str).str.lower() \
#              .str.contains("mercadopago|tiktok|sindelantal|gardenscapes|dalvik|wish|trivago|grability") == True].value_counts()

In [11]:
installs = installs.drop(columns=['kind', 'user_agent', 'trans_id'])

# **Ventanas para train y test**

In [12]:
time_start_1 = '2019-04-18'
time_start_2 = '2019-04-19'
time_start_3 = '2019-04-20'
time_start_4 = '2019-04-21'
time_start_5 = '2019-04-22'

In [13]:
# Ventanas para train y testing

window_1 = installs.loc[installs['created'].between('2019-04-18', '2019-04-21')]
window_2 = installs.loc[installs['created'].between('2019-04-19', '2019-04-22')]
window_3 = installs.loc[installs['created'].between('2019-04-20', '2019-04-23')]
# window_4 = installs.loc[installs['created'].between('2019-04-21', '2019-04-24')]
# window_5 = installs.loc[installs['created'].between('2019-04-22', '2019-04-25')]

In [14]:
# Ordeno por fechas

window_1 = window_1.sort_values('created')
window_2 = window_2.sort_values('created')
window_3 = window_3.sort_values('created')
# window_4 = window_4.sort_values('created')
# window_5 = window_5.sort_values('created')

In [15]:
# window_1.head()

In [16]:
# Contar la cantidad de apariciones que tiene un usuario por ventada

window_1['count_installs'] = window_1.groupby('ref_hash')['ref_hash'].transform('count')
window_2['count_installs'] = window_2.groupby('ref_hash')['ref_hash'].transform('count')
window_3['count_installs'] = window_3.groupby('ref_hash')['ref_hash'].transform('count')
# window_4['count_installs'] = window_4.groupby('ref_hash')['ref_hash'].transform('count')
# window_5['count_installs'] = window_5.groupby('ref_hash')['ref_hash'].transform('count')

In [17]:
# primera y ultima instalacion por ref_hash

window_1['last_install'] = window_1.groupby('ref_hash')['created'].transform('max')
window_1['first_install'] = window_1.groupby('ref_hash')['created'].transform('min')

window_2['last_install'] = window_2.groupby('ref_hash')['created'].transform('max')
window_2['first_install'] = window_2.groupby('ref_hash')['created'].transform('min')

window_3['last_install'] = window_3.groupby('ref_hash')['created'].transform('max')
window_3['first_install'] = window_3.groupby('ref_hash')['created'].transform('min')

# window_4['last_install'] = window_4.groupby('ref_hash')['created'].transform('max')
# window_4['first_install'] = window_4.groupby('ref_hash')['created'].transform('min')

# window_5['last_install'] = window_5.groupby('ref_hash')['created'].transform('max')
# window_5['first_install'] = window_5.groupby('ref_hash')['created'].transform('min')

In [18]:
# diferencia de tiempo entre la primera y la ultima instalacion

window_1['diff_between_installs'] = (window_1['last_install'] - window_1['first_install']).dt.total_seconds()

window_2['diff_between_installs'] = (window_2['last_install'] - window_2['first_install']).dt.total_seconds()

window_3['diff_first_installs'] = (window_3['last_install'] - window_3['first_install']).dt.total_seconds()

# window_4['diff_first_installs'] = (window_4['last_install'] - window_4['first_install']).dt.total_seconds()

# window_5['diff_first_installs'] = (window_5['last_install'] - window_5['first_install']).dt.total_seconds()

In [19]:
# Promedio de diferencias de tiempo entre todas las instalaciones
window_1['prom_time_install'] = 0
window_1.loc[window_1['count_installs'] > 1, 'prom_time_install'] = ((window_1['last_install'] - window_1['first_install'])/ \
                                 (window_1['count_installs'] -1)).dt.total_seconds()

window_2['prom_time_install'] = 0
window_2.loc[window_2['count_installs'] > 1,'prom_time_install'] = ((window_2['last_install'] - window_2['first_install'])/ \
                                 (window_2['count_installs'] - 1)).dt.total_seconds()

# window_3['prom_time_install'] = 0
window_3.loc[window_3['count_installs'] > 1,'prom_time_install'] = ((window_3['last_install'] - window_3['first_install'])/ \
                                 (window_3['count_installs'] - 1)).dt.total_seconds()

# window_4['prom_time_install'] = 0
# window_4.loc[window_4['count_installs'] > 1, 'prom_time_install'] = ((window_4['last_install'] - window_4['first_install'])/ \
#                                  (window_4['count_installs'] - 1)).dt.total_seconds()

# window_5['prom_time_install'] = 0
# window_5.loc[window_5['count_installs'] > 1, 'prom_time_install'] = ((window_5['last_install'] - window_5['first_install'])/ \
#                                  (window_5['count_installs'] -1 )).dt.total_seconds()

In [20]:
# diferencia de tiempo entre el comienzo de la ventana y la primera y ultima instalacion

window_1['time_first_install'] = (window_1['first_install'] - pd.Timestamp(time_start_1)).dt.total_seconds()
window_1['time_last_install'] = (window_1['last_install'] - pd.Timestamp(time_start_1)).dt.total_seconds()

window_2['time_first_install'] = (window_2['first_install'] - pd.Timestamp(time_start_2)).dt.total_seconds()
window_2['time_last_install'] = (window_2['last_install'] - pd.Timestamp(time_start_2)).dt.total_seconds()

window_3['time_first_install'] = (window_3['first_install'] - pd.Timestamp(time_start_3)).dt.total_seconds()
window_3['time_last_install'] = (window_3['last_install'] - pd.Timestamp(time_start_3)).dt.total_seconds()

# window_4['time_first_install'] = (window_4['first_install'] - pd.Timestamp(time_start_4)).dt.total_seconds()
# window_4['time_last_install'] = (window_4['last_install'] - pd.Timestamp(time_start_4)).dt.total_seconds()

# window_5['time_first_install'] = (window_5['first_install'] - pd.Timestamp(time_start_5)).dt.total_seconds()
# window_5['time_last_install'] = (window_5['last_install'] - pd.Timestamp(time_start_5)).dt.total_seconds()

In [21]:
window_1 = window_1.groupby('ref_hash').head(1)
window_2 = window_2.groupby('ref_hash').head(1)
window_3 = window_3.groupby('ref_hash').head(1)
# window_4 = window_4.groupby('ref_hash').head(1)
# window_5 = window_5.groupby('ref_hash').head(1)

In [22]:
print(len(window_1))
print(len(window_2))
print(len(window_3))
# print(len(window_4))
# print(len(window_5))

132297
129651
131181


In [23]:
window_1 = window_1.drop(columns=['created', 'last_install', 'first_install'])
window_2 = window_2.drop(columns=['created', 'last_install', 'first_install'])
window_3 = window_3.drop(columns=['created', 'last_install', 'first_install'])
# window_4 = window_4.drop(columns=['created', 'last_install', 'first_install'])
# window_5 = window_5.drop(columns=['created', 'last_install', 'first_install'])

# **Ventatas 1 y 2**

In [24]:
# primer join 
join1 = window_1.merge(window_2[['ref_hash','time_first_install']],indicator = False, how='left', on='ref_hash')

In [ ]:
print(len(join1), len(window_1), len(window_2))

132297 132297 129651


In [25]:
join1 = join1.drop(columns=['ref_hash'])
join1['time_first_install_y'].fillna(20*24*3600, inplace=True) # 20 dias, tiempo muy grande

In [ ]:
join1.head()

,application_id,attributed,implicit,wifi,ref_type_android,ref_type_iphone,weekday_0,weekday_1,weekday_2,weekday_3,...,agent_wish,agent_trivago,agent_grability,agent_mercadopago,count_installs,diff_between_installs,prom_time_install,time_first_install_x,time_last_install,time_first_install_y
0,70,0,0,-1.0,1,0,0,0,0,1,...,0,0,0,0,1,0.000,0.000,1.560,1.560,1728000.0
1,70,0,0,-1.0,1,0,0,0,0,1,...,0,0,0,0,1,0.000,0.000,1.851,1.851,1728000.0
2,65,0,1,0.0,1,0,0,0,0,1,...,0,0,0,0,2,4.006,4.006,5.152,9.158,1728000.0
3,27,0,0,1.0,0,1,0,0,0,1,...,0,1,0,0,1,0.000,0.000,5.589,5.589,1728000.0
4,339,0,0,1.0,0,1,0,0,0,1,...,0,0,0,0,1,0.000,0.000,6.795,6.795,1728000.0


### **XGBOOX**

In [26]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [27]:
mod1 = xgb.XGBRegressor(gamma=1, learning_rate=0.01, max_depth=3, n_estimators=1000, subsample=0.8,
                        random_state=34) 

In [28]:
mod2 = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10000)

In [29]:
train1, test1 = train_test_split(join1, test_size=0.2)

train1_Y = train1['time_first_install_y']
train1_X = train1.drop(columns=['time_first_install_y'])
test1_Y = test1['time_first_install_y']
test1_X = test1.drop(columns=['time_first_install_y'])

In [30]:
mod1.fit(train1_X,train1_Y)

preds1_join1 = mod1.predict(test1_X)

/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[23:12:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
rmse1_join1 = np.sqrt(mean_squared_error(test1_Y, preds1_join1))
print("RMSE: %f" % (rmse1_join1))

RMSE: 79606.431133


In [32]:
mod2.fit(train1_X,train1_Y)

preds2_join1 = mod2.predict(test1_X)

/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[23:14:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [33]:
rmse2_join1 = np.sqrt(mean_squared_error(test1_Y, preds2_join1))
print("RMSE: %f" % (rmse2_join1))

RMSE: 107321.237725


### **Random Forest**

In [34]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [35]:
rf.fit(train1_X, train1_Y);

In [36]:
preds3_join1 = rf.predict(test1_X)

In [37]:
rmse3_join1 = np.sqrt(mean_squared_error(test1_Y, preds3_join1))

print("RMSE: %f" % (rmse3_join1))

RMSE: 88916.159368


# **Ventana 2 y 3**

In [ ]:
# pd.concat([df1, preds3_join1], axis=1)

In [ ]:
join2 = window_2.merge(window_3[['ref_hash','time_first_install']],indicator = False, how='left', on='ref_hash')

In [ ]:
train2, test2 = train_test_split(join2, test_size=0.2)

train2_Y = train2['time_first_install_y']
train2_X = train2.drop(columns=['time_first_install_y'])
test2_Y = test2['time_first_install_y']
test2_X = test2.drop(columns=['time_first_install_y'])

### Clicks

In [ ]:
# clicks = pd.read_csv("data/clicks.csv.gzip")

# clicks = clicks.drop(columns=['agent_device', 'os_minor', 'os_major', 'country_code', 'action_id'\
#                 'brand', 'touchX', 'touchY', 'specs_brand', 'carrier_id', 'longitude', 'latitude'], axis=1)

In [ ]:
# clicks.head()

In [ ]:
# installs['wifi'] = installs['wifi'].map({True: 1, False: 0})
# installs['wifi'] = installs['wifi'].fillna(-1)

# installs['ref_type'] = installs['ref_type'].map({'1891515180541284343': 1, '1494519392962156891': 0})

In [ ]:
# window_1_click = clicks[clicks['created'].between('2019-04-18', '2019-04-21')]

In [ ]:
# clicks_first = pd.merge(window_1_click, window_1_click.groupby('ref_hash').agg('size').reset_index(), on='ref_hash', how='inner')

In [ ]:
# clicks_first = clicks_first.drop_duplicates('ref_hash')

In [ ]:
# installs_ref_hash_2 = pd.merge(installs, clicks, on='ref_hash', how='inner')

In [ ]:
# installs_ref_hash_2.head()